In [1]:
import pandas as pd
from copy import deepcopy
original_df = pd.read_csv('stage1/input/forSurv.csv')

In [7]:
cols = original_df.columns.tolist()
cols.remove('BSPT_SRV')
cols.remove('OVRL_SRVL_DTRN_DCNT')

cols

['BSPT_IDGN_AGE',
 'BSPT_SEX_CD',
 'C18',
 'C19',
 'C20',
 'IMPT_HM1E_RSLT_CD',
 'IMPT_HS2E_RSLT_CD',
 'BPTH_RSLT',
 'CEA',
 'MLPT_MSIE_RSLT_CD',
 'MLPT_KRES_RSLT_CD',
 'MLPT_NREX_RSLT_CD',
 'OPRT_CNTS',
 'TRTN_CNT']

In [32]:
from lifelines import CoxPHFitter 

cph = CoxPHFitter(penalizer=0.01)
check_book = {}
for var in cols:
    cph.fit(df=original_df, duration_col='OVRL_SRVL_DTRN_DCNT',event_col='BSPT_SRV',formula=f'{var}')
    check_book[var] = (cph.summary.p.values.item() < 0.05)
    

/home/wonseok/synthetic_data/.env/lib64/python3.6/site-packages/lifelines/fitters/coxph_fitter.py:1602: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  "Newton-Rhaphson failed to converge sufficiently. {0}".format(CONVERGENCE_DOCS), exceptions.ConvergenceWarning


In [33]:
check_book

{'BSPT_IDGN_AGE': True,
 'BSPT_SEX_CD': False,
 'C18': False,
 'C19': False,
 'C20': False,
 'IMPT_HM1E_RSLT_CD': False,
 'IMPT_HS2E_RSLT_CD': False,
 'BPTH_RSLT': False,
 'CEA': True,
 'MLPT_MSIE_RSLT_CD': False,
 'MLPT_KRES_RSLT_CD': False,
 'MLPT_NREX_RSLT_CD': False,
 'OPRT_CNTS': True,
 'TRTN_CNT': True}

# 변수 선정
변수를 결국 줄인다. 우리는 BSPT_SEX_CD, CEA, OPRT_CNTS, TRTN_CNT로 진행한다

In [36]:
parsed_original_df = original_df[['BSPT_SRV','OVRL_SRVL_DTRN_DCNT','BSPT_IDGN_AGE','CEA','OPRT_CNTS','TRTN_CNT']]

In [37]:
df1 = deepcopy(parsed_original_df.sample(frac=0.5,random_state=1))
df2 = deepcopy(parsed_original_df.iloc[~original_df.index.isin(df1.index)])

df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

df1.to_csv('stage1/input/forSurv1.csv',index=False)
df2.to_csv('stage1/input/forSurv2.csv', index=False)

In [20]:
pd.read_csv('stage1/input/forSurv1.csv').sample(1)

,BSPT_SRV,OVRL_SRVL_DTRN_DCNT,BSPT_IDGN_AGE,CEA,OPRT_CNTS,TRTN_CNT
2855,1,138,59,6.28,0.0,1.0


3개의 AUC 및 predicted beta 보여줄 것
- 모든 데이터에 대해서 prediction이 가능함. --> 본래 cox 와 비교하는 plot